In [1]:
import os

# Path to the directory containing the labels
labels_directory = 'E:\\datasets\\'

# List to store filenames of changed image files
changed_image_files = []

# Iterate over the labels subdirectories
for root, dirs, files in os.walk(labels_directory):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(root, file)
            image_file = os.path.splitext(file)[0] + '.jpg'  # Assuming the images have the same name but with a different extension

            with open(file_path, 'r') as f:
                lines = f.readlines()

            # Modify the YOLO bounding box coordinates in each line
            modified_lines = []
            for line in lines:
                values = line.strip().split(' ')
                class_label = values[0]
                modified_values = [class_label]

                for i in range(1, len(values)):
                    value = float(values[i])
                    if i % 2 == 1:  # X-coordinate or width
                        if value < 0:
                            value = 0
                        elif value > 1:
                            value = 1
                    else:  # Y-coordinate or height
                        if value < 0:
                            value = 0
                        elif value > 1:
                            value = 1
                    modified_values.append(str(value))

                modified_line = ' '.join(modified_values) + '\n'
                modified_lines.append(modified_line)

            # Write the modified lines back to the file
            with open(file_path, 'w') as f:
                f.writelines(modified_lines)

            changed_image_files.append(image_file)

# Write the list of changed image files to a separate file
changed_files_path = os.path.join(labels_directory, 'changed_image_files.txt')
with open(changed_files_path, 'w') as f:
    f.write('\n'.join(changed_image_files))

print(f"Total changed image files: {len(changed_image_files)}")
print(f"List of changed image files saved to: {changed_files_path}")


Total changed image files: 51604
List of changed image files saved to: E:\datasets\changed_image_files.txt


In [16]:
import os
import random
from PIL import Image, ImageDraw

# Path to the directory containing the images
images_directory = 'E:\\datasets\\zooniverse\\images\\val\\'

# Path to the directory containing the labels
labels_directory = 'E:\\datasets\\zooniverse\\labels\\val\\'

# Choose a random image
image_files = os.listdir(images_directory)
random_image_file = random.choice(image_files)
image_path = os.path.join(images_directory, random_image_file)

# Corresponding label file
label_file = os.path.splitext(random_image_file)[0] + '.txt'
label_path = os.path.join(labels_directory, label_file)

# Load the image
image = Image.open(image_path)
draw = ImageDraw.Draw(image)

# Read the label file
with open(label_path, 'r') as f:
    lines = f.readlines()

# Draw bounding boxes on the image
for line in lines:
    class_id, x, y, width, height = map(float, line.strip().split(' '))
    left = (x - width / 2) * image.width
    top = (y - height / 2) * image.height
    right = (x + width / 2) * image.width
    bottom = (y + height / 2) * image.height
    draw.rectangle([(left, top), (right, bottom)], outline='red')

# Show the image with labels
image.show()

